In [333]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import re
from datetime import datetime
import os
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [334]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

## 변수 설정

In [335]:
query = input("검색어를 입력하세요 : ")
news_num = int(input("크롤링할 기사 개수를 입력하세요 : "))

In [336]:
print(query, ',' ,news_num)

코로나 , 10


## 페이지 읽어오기

In [337]:
base_url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={query}'

In [338]:
news_dict = {}
idx = 0
page = 1
contents = []

In [339]:
response = requests.get(base_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [351]:
for info in soup.select('a.info') :
    if info.text == '네이버뉴스' :
        print(info['href'])

https://n.news.naver.com/mnews/article/001/0013893547?sid=104
https://n.news.naver.com/mnews/article/366/0000895136?sid=105
https://n.news.naver.com/mnews/article/015/0004835101?sid=101
https://n.news.naver.com/mnews/article/023/0003758960?sid=101
https://n.news.naver.com/mnews/article/052/0001874974?sid=104
https://n.news.naver.com/mnews/article/003/0011811429?sid=100
https://n.news.naver.com/mnews/article/047/0002389428?sid=102
https://n.news.naver.com/mnews/article/023/0003758681?sid=101
https://n.news.naver.com/mnews/article/029/0002795814?sid=101
https://n.news.naver.com/mnews/article/584/0000022745?sid=103


In [341]:
print('기사 수집 중입니다.')
while idx < news_num :   
    
    response = requests.get(base_url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    news_list = soup.select('div.news_area')
    next_page = soup.select('div.sc_page_inner > a.btn')

    for news in news_list :
        if news.select('a.info')[-1].text == '네이버뉴스' :
            link = news.select('a.info')[-1]['href']
            title = news.select_one('a.news_tit').text
            text = news.select_one('a.api_txt_lines.dsc_txt_wrap').text
            count = news.find("span", {"class" : "u_cbox_info_txt"}).get_text()

            driver = webdriver.Chrome('chromedriver.exe')
            driver.get(link)
            time.sleep(3)
            driver.find_element(By.XPATH, './/*[@id="cbox_module"]/div[2]/div[9]/a/span[1]').click()
            html2 = driver.page_source
            soup2 = BeautifulSoup(html2, 'lxml')
            comment_list = soup2.find_all("span", {'class' : "u_cbox_contents"})

            for comment in comment_list :
                comment = comment.text
                contents.append(comment)

            news_dict[idx] = {'제목' : title, '링크' : link, '본문 요약' : text, '댓글' : contents}
            idx += 1
        else :
            continue

    page += 1
    
    for p in next_page :
        if p.text == str(page) :
            news_url = f'https://search.naver.com/search.naver' + p['href']

print('크롤링이 완료되었습니다.')

기사 수집 중입니다.


C:\Users\Playdata\AppData\Local\Temp\ipykernel_11292\1273187364.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=112.0.5615.137)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B0DCE3+50899]
	(No symbol) [0x00A9E111]
	(No symbol) [0x009A5450]
	(No symbol) [0x009D1846]
	(No symbol) [0x009C94A5]
	(No symbol) [0x009EB8FC]
	(No symbol) [0x009C8EC6]
	(No symbol) [0x009EBC54]
	(No symbol) [0x009FE01C]
	(No symbol) [0x009EB6F6]
	(No symbol) [0x009C7708]
	(No symbol) [0x009C886D]
	GetHandleVerifier [0x00D73EAE+2566302]
	GetHandleVerifier [0x00DA92B1+2784417]
	GetHandleVerifier [0x00DA327C+2759788]
	GetHandleVerifier [0x00BA5740+672048]
	(No symbol) [0x00AA8872]
	(No symbol) [0x00AA41C8]
	(No symbol) [0x00AA42AB]
	(No symbol) [0x00A971B7]
	BaseThreadInitThunk [0x75E97D49+25]
	RtlInitializeExceptionChain [0x770CB74B+107]
	RtlClearBits [0x770CB6CF+191]


In [ ]:
df = pd.DataFrame(news_dict).T
df